In [ ]:
import numpy as np

from downstream_models import MF, PLRec

from utils import get_data, ndcg, recall

from copy import deepcopy
from bayes_opt import BayesianOptimization

In [ ]:
seed = 1337
np.random.seed(seed)

In [ ]:
dict_map = lambda d, f: {x: ((f(y[0]), f(y[1])) if isinstance(y, tuple) else f(y)) for x, y in d.items()}

In [ ]:
data = get_data('<specify your data directory here>')
train_data, valid_in_data, valid_out_data, test_in_data, test_out_data = data
n_users, n_items = train_data.shape

In [ ]:
def generate(batch_size, device, data_in, data_out=None, shuffle=False, samples_perc_per_epoch=1):
    assert 0 < samples_perc_per_epoch <= 1
    
    total_samples = data_in.shape[0]
    samples_per_epoch = int(total_samples * samples_perc_per_epoch)
    
    if shuffle:
        idxlist = np.arange(total_samples)
        np.random.shuffle(idxlist)
        idxlist = idxlist[:samples_per_epoch]
    else:
        idxlist = np.arange(samples_per_epoch)
    
    for st_idx in range(0, samples_per_epoch, batch_size):
        end_idx = min(st_idx + batch_size, samples_per_epoch)
        idx = idxlist[st_idx:end_idx]

        yield Batch(device, idx, data_in, data_out)


class Batch:
    def __init__(self, device, idx, data_in, data_out=None):
        self._device = device
        self._idx = idx
        self._data_in = data_in
        self._data_out = data_out
    
    def get_idx(self):
        return self._idx
    
    def get_idx_to_dev(self):
        return torch.LongTensor(self.get_idx()).to(self._device)
        
    def get_ratings(self, is_out=False):
        data = self._data_out if is_out else self._data_in
        return data[self._idx]
    
    def get_ratings_to_dev(self, is_out=False):
        return torch.Tensor(
            self.get_ratings(is_out).toarray()
        ).to(self._device)


In [ ]:
test_metrics = [{'metric': ndcg, 'k': 100}, {'metric': recall, 'k': 20}, {'metric': recall, 'k': 50}]

def evaluate(model, data_in, data_out, metrics, model_kwargs, samples_perc_per_epoch=1, batch_size=2000):
    metrics = deepcopy(metrics)
    
    for m in metrics:
        m['score'] = []
    
    for batch in generate(batch_size=batch_size,
                          device=None,
                          data_in=data_in,
                          data_out=data_out,
                          samples_perc_per_epoch=samples_perc_per_epoch
                         ):
        
        ratings_in = batch.get_ratings()
        ratings_out = batch.get_ratings(is_out=True)
    
        ratings_pred = model.predict(ratings_in)
        
        if not (data_in is data_out):
            ratings_pred[batch.get_ratings().nonzero()] = -np.inf
            
        for m in metrics:
            m['score'].append(m['metric'](ratings_pred, ratings_out, k=m['k']))

    for m in metrics:
        m['score'] = np.concatenate(m['score']).mean()
        
    return [x['score'] for x in metrics]


In [ ]:
best_score_ever = -np.inf

In [ ]:
def run(fixed_params, optimizing_params):
    global best_score_ever
    
    optimizing_params = dict_map(optimizing_params, np.exp)
    print(optimizing_params)
    
    params = fixed_params | optimizing_params

    Model = params['model']
    model = Model(train_data, params)

    best_score = -np.inf
    
    # learning
    
    for epoch in range(-params['max_init_epochs'], params['max_train_epochs']):
        
        if epoch < 0:
            model.init()
            if params['max_train_epochs'] > 0:
                continue
        else:
            if params['init_after_step']:
                model.init()
            model.step()

        score = evaluate(model, valid_in_data, valid_out_data, [{'metric': ndcg, 'k': 100}], params, 1)[0]

        print(score)

        if score > best_score + 1e-4:
            best_score = score
            Q_best = model.Q.copy()
            W_best = model.W.copy() if hasattr(model, 'W') else None
        else:
            break


    # evaluaton
    if best_score > best_score_ever:
        best_score_ever = best_score
        model.Q = Q_best
        model.W = W_best
        final_scores = evaluate(model, test_in_data, test_out_data, test_metrics, params)
        for metric, score in zip(test_metrics, final_scores):
            print(f"{metric['metric'].__name__}@{metric['k']}:\t{score:.4f}")
        for metric, score in zip(test_metrics, final_scores):
            print(f"{score:.3f}")

    
    return best_score

In [ ]:
# MF init + reg setup
pbounds = {'item_λ': (1e1, 1e3), 'item_α': (1e-2, 1e1), 'item_thr': (1e0, 1e3),
           'r_q':  (1e-1, 1e2), 's_q':  (1e-1, 1e3), 'r_p':  (1e-1, 1e3)
          }
probe_optimizing_params = {'item_thr': 264.76225833649335,
                           'item_α': 0.8134979262528148,
                           'item_λ': 944.9124867577675,
                           'r_p': 1606.1732584049885,
                           'r_q': 0.0002834483496119082,
                           's_q': 26.91023013144844, 
                          }
fixed_params = {'model': MF, 
                'L': 256, 'bias': True,
                'max_init_epochs': 1, 'max_train_epochs': 10,
                'init': 'ImplicitSLIM', 'reg': 'ImplicitSLIM',
                'orth': False, 'init_after_step': False, 
               }

# # PLRec + LLE-SLIM
# pbounds = {'r_q':  (1e-3, 1e3), 'power':  (1, 10), 'item_λ': (1e2, 1e5)}
# probe_optimizing_params = {'item_λ': 1218.249396070337, 
#                            'power': 2.8606363101792773, 
#                            'r_q': 19.64194050012942,
#                           }
# fixed_params = {'model': PLRec,
#                 'L': 128, 'bias': False, 
#                 'max_init_epochs': 1, 'max_train_epochs': 1,
#                 'init': 'LLE-SLIM', 'reg': 'none',
#                 'orth': False, 'init_after_step': False, 
#                }

# # PLRec
# pbounds = {'r_q':  (1e-3, 1e3), 'power':  (1, 10)}
# probe_optimizing_params = {'power': 1.2778018325012814, 
#                            'r_q': 0.008348500653807706,
#                           }
# fixed_params = {'model': PLRec,
#                 'L': 256, 'bias': False, 
#                 'max_init_epochs': 1, 'max_train_epochs': 1,
#                 'init': 'SVD', 'reg': 'none',
#                 'orth': False, 'init_after_step': False, 
#                }

# # MF
# pbounds = {'r_p':  (1e-3, 1e3), 'r_q':  (1e-3, 1e3)}
# probe_optimizing_params = {'r_p': 19.804162868530003,
#                            'r_q': 185.54338507706711,
#                           }
# fixed_params = {'model': MF,
#                 'L': 256, 'bias': True, 
#                 'max_init_epochs': 0, 'max_train_epochs': 20,
#                 'init': 'none', 'reg': 'none',
#                 'orth': False, 'init_after_step': False, 
#                }

# # MF + LLE-SLIM
# pbounds = {'r_p':  (1e-3, 1e3), 'power':  (1, 10), 'item_λ': (1e2, 1e5)}
# probe_optimizing_params = {'item_λ': 4470.118449330024, 
#                            'power': 4.489624436421516, 
#                            'r_p': 0.15477826213462,
#                           }
# fixed_params = {'model': MF,
#                 'L': 256, 'bias': True, 
#                 'max_init_epochs': 1, 'max_train_epochs': 0,
#                 'init': 'LLE-SLIM', 'reg': 'none',
#                 'orth': False, 'init_after_step': False, 
#                }

# # MF multistep init setup
# pbounds = {'item_λ': (1e1, 1e4), 'item_α': (1e-4, 1e1), 'item_thr': (1e0, 1e3),
#            'r_p':  (1e-3, 1e3)
#           }
# probe_optimizing_params = {'item_thr': 394.82499101128354, 
#                            'item_α': 0.5176648566904536, 
#                            'item_λ': 597.4182355801788, 
#                            'r_p': 1414.4673175122664, 
#                           }
# fixed_params = {'model': MF, 
#                 'L': 256, 'bias': True,
#                 'max_init_epochs': 10, 'max_train_epochs': 0,
#                 'init': 'ImplicitSLIM', 'reg': 'none',
#                 'orth': False, 'init_after_step': False, 
#                }

# # PLRec multistep init setup
# pbounds = {'item_λ': (1e0, 1e4), 'item_α': (1e-3, 1e+1), 'item_thr': (1e0, 1e2),
#            'r_q':  (1e-3, 1e3), 's_q':  (1e-3, 1e5), 'power':  (1, 1e5)
#           }
# probe_optimizing_params = {'item_thr': 82.39948654917328, 
#                            'item_α': 10.000000000000002, 
#                            'item_λ': 907.3171013771947, 
#                            'power': 10000.00000000001, 
#                            'r_q': 7.826179962320708, 
#                            's_q': 164074.31862023394, 
#                           }
# fixed_params = {'model': PLRec, 
#                 'L': 256, 'bias': False,
#                 'max_init_epochs': 0, 'max_train_epochs': 10,
#                 'init': 'ImplicitSLIM', 'reg': 'ImplicitSLIM',
#                 'orth': True, 'init_after_step': True, 
#                }

In [ ]:
run_wrapper = lambda **optimizing_params: run(fixed_params, optimizing_params)

optimizer = BayesianOptimization(
    f=run_wrapper,
    pbounds=dict_map(pbounds, np.log),
    verbose=2,
    random_state=1,
)

optimizer.probe(
    params=dict_map(probe_optimizing_params, np.log),
    lazy=True,
)

optimizer.maximize(
    init_points=0,
    n_iter=0,
)